In [1]:
import sys
sys.path.append("../")
from functools import partial

import torch
from torch.utils.data import DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

from src.models.rnn_model import RNN
from src.models.learned_masking import DeepAugmenter
from src.data.dataio import DataFiles, Dataset, truncate_fn

transformers.logging.set_verbosity_error()

In [2]:
BERT_MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=BERT_MODEL_NAME)

data_files = DataFiles.from_dir(
    "/home/pavlo/comp-550/comp-550-project/data/articles/clean_articles/training"
)

train_dataset = Dataset(data_files)
# train_dataset = train_dataset.map(
#     partial(
#         truncate_fn, 
#         tokenizer=tokenizer, 
#         max_seq_length=128, 
#         fill_to_max=False
#     )
# )


Resolving data files:   0%|          | 0/1780 [00:00<?, ?it/s]

Using custom data configuration default-6574999eee52b4d9
Reusing dataset text (/home/pavlo/.cache/huggingface/datasets/text/default-6574999eee52b4d9/0.0.0/e10c27cd9493b3e5d594862e2a0d24e29034e10b895b72057984ed60b5a1c333)


In [3]:
# data_loader = DataLoader(train_dataset, batch_size=10)

model = DeepAugmenter()

In [ ]:
model.train(train_dataset)